In [13]:
import pandas as pd

strDate = '20200801'
endDate = '20210801'

dt_index = pd.date_range(start=strDate, end=endDate)
dt_list = dt_index.strftime("%Y%m%d").tolist()

df = pd.DataFrame()
list = []
for i in dt_list:
    for j in range(1,11):
        list.append([i, j])

df = pd.DataFrame(list)
df

,0,1
0,20200801,1
1,20200801,2
2,20200801,3
3,20200801,4
4,20200801,5
...,...,...
3655,20210801,6
3656,20210801,7
3657,20210801,8
3658,20210801,9


In [4]:
import sqlite3

conn = sqlite3.connect('kr_scrapping.db')
c = conn.cursor()

c.execute('DROP TABLE IF EXISTS `news_words`;')
c.execute('''
        CREATE TABLE `news_words` (
            `words_id` INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
            `pub_date` TEXT NOT NULL,
            `pub_time` TEXT NOT NULL,
            `news_words` TEXT NOT NULL,
            `news_id` INTEGER NOT NULL);
            ''')


# c.execute('ALTER TABLE `test_dummy` ADD COLUMN words_id INTEGER REFERENCES test_words(words_id);')
# c.execute('''ALTER TABLE `test_dummy` ADD CONSTRAINT
#                   FOREIGN KEY (test_words) 
#                   REFERENCES test_words(words_id);''')
conn.commit()
conn.close()

In [10]:
conn = sqlite3.connect('test.db')
c = conn.cursor()
c.execute("SELECT * FROM test_words")

In [5]:
import sqlite3
from konlpy.tag import Okt
import re

conn = sqlite3.connect('kr_scrapping.db')
c = conn.cursor()
c.execute("SELECT news_id, pubdate, news_content FROM news_dummy")

okt = Okt()

news_nouns = []
for i in c.fetchall():
  news_id = i[0]
  pubdate = re.sub('[. ]','', i[1])[:8]
  pubtime = i[1][-5:]
  news_content = i[2]
  news_word = okt.nouns(news_content)  # okt를 사용하여 명사 단위로 끊음
  words = []
  for prep_word in news_word:  # exist_word 리스트 안의 단어를 하나씩 뽑아서 반복
    if len(prep_word) > 1:  # 뽑아낸 단어 중 2글자 이상인 것
      words.append(prep_word)
  # print([pubdate, pubtime, words])
  c.execute("INSERT INTO news_words (pub_date, pub_time, news_words, news_id) VALUES(?,?,?,?)", (pubdate, pubtime, str(words), news_id))

conn.commit()
conn.close()

In [ ]:
import sqlite3
import re
from collections import Counter

conn = sqlite3.connect('kr_scrapping.db')
c = conn.cursor()
c.execute("SELECT news_words FROM news_ WHERE pub_time LIKE '23:2%'")

news_words = c.fetchall()

key_words = []
for words in news_words:
    words = re.sub("[-=+#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]","", str(words))
    words = words.split(', ')
    for word in words:
        word = word.strip()
        key_words.append(word)

# key_words
cnt_2 = Counter(key_words)
# cnt
conn.close()
sorted(cnt_2.items(), key = lambda item: item[1], reverse=True)